# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [2]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        degree = 2
        a = 1
        kernel = (kernel + a)**degree
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [3]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

iris = load_iris()
data_set = iris.data
labels = iris.target

In [4]:
def choose_set_for_label(data_set, label):

    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

    filtered_train = np.ones(len(train_labels)) * -1
    filtered_train[train_labels == label] = 1
    train_labels = filtered_train

    filtered_test = np.ones(len(test_labels)) * -1
    filtered_test[test_labels == label] = 1
    test_labels = filtered_test

    return train_data_set, test_data_set, train_labels, test_labels

In [5]:
_, _, train_labels, test_labels = choose_set_for_label(data_set, 1)
train_labels

array([-1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
        1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,
       -1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,
       -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,
       -1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,
       -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1., -1.])

In [6]:
# General test_labels : 
general_train_data_set, general_test_data_set, general_train_labels, general_test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

general_train_labels

array([0, 2, 1, 2, 0, 0, 2, 0, 0, 1, 2, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2,
       2, 1, 1, 1, 0, 1, 2, 2, 2, 2, 0, 2, 1, 2, 1, 0, 1, 0, 0, 2, 1, 1,
       1, 1, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 1, 1, 2, 1, 0, 0, 1, 1, 1, 0,
       2, 0, 1, 0, 1, 0, 2, 2, 2, 0, 1, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 2,
       0, 1, 1, 2, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2,
       0, 1, 0, 0, 2, 1, 2, 2, 2, 2])

In [7]:
def get_labels_count(data_set):
    unique_labels_count = len(set(labels))
    return unique_labels_count

In [ ]:
Use the code that we have implemented earlier:

In [25]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [27]:
predicted = [0]*(int(0.2*len(labels)))
print("Predicted labels : ")
print(predicted)

# General test_labels : 
general_train_data_set, general_test_data_set, general_train_labels, general_test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

for label in range(3):
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, label)

    objects_count = len(train_labels)

    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted_iter = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted_iter = list(predicted_iter.astype(int))

    print(f"Label : {label}")
    print(f"Predictions for label {label}")
    print(predicted_iter)

    n = len(predicted_iter)
    for i in range(n):
        if predicted_iter[i] == 1:
            predicted[i] = label

    print(f"Predicted labels at step label {label}")
    print(predicted)
    print('Actual labels')
    print(general_test_labels)

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, general_test_labels))

Predicted labels : 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
     pcost       dcost       gap    pres   dres
 0:  1.1726e+02 -1.7992e+03  4e+03  2e-01  2e-15
 1:  7.7967e+01 -1.6973e+02  3e+02  5e-03  2e-15
 2:  1.0169e+01 -2.2323e+01  3e+01  2e-16  3e-15
 3: -4.9607e-01 -4.9938e+00  4e+00  5e-16  1e-15
 4: -1.4344e+00 -2.5218e+00  1e+00  2e-16  4e-16
 5: -1.6897e+00 -2.1674e+00  5e-01  2e-16  2e-16
 6: -1.8078e+00 -2.0073e+00  2e-01  2e-16  2e-16
 7: -1.8464e+00 -1.9763e+00  1e-01  8e-16  1e-16
 8: -1.8825e+00 -1.9020e+00  2e-02  2e-16  2e-16
 9: -1.8902e+00 -1.8906e+00  4e-04  3e-16  2e-16
10: -1.8904e+00 -1.8904e+00  5e-06  4e-16  2e-16
11: -1.8904e+00 -1.8904e+00  5e-08  7e-16  2e-16
Optimal solution found.
Label : 0
Predictions for label 0
[array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), ar